# Notebook
This is just a way of making it easier to go through the audio features, and share any work that I've been doing. This is done by Nicolas

This will probably be a bit messy, but it exists and will be commented and that's what matters

In [29]:
import pandas as pd
import numpy as np
from model_trainer_and_tester import *
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Embedding, Input, TimeDistributed
from model.scoring_metrics import get_windiff, get_pk, get_k_kappa

# train_set = read_in_dataset([], 'train')
#
# test_set = read_in_dataset([], 'test')

features = ['pause','speakerChange', 'similarity', 'f0_diff', 'f0_baseline_diff']
shift = [-1, 0, 1]

X_train, Y_train = read_in_dataset_lstm(features, shift, to_read='train')
X_test, Y_test = read_in_dataset_lstm(features, shift, to_read='test')

X_train = X_train.astype(float)
X_test = X_test.astype(float)

D:\Programming\University\social-signal-processing-project\source\model_trainer_and_tester.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_y.iloc[-1] = 1.0
D:\Programming\University\social-signal-processing-project\source\model_trainer_and_tester.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_y.iloc[-1] = 1.0
D:\Programming\University\social-signal-processing-project\source\model_trainer_and_tester.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [32]:
batch_size = 64
n_timesteps = 3
train_ratio = 0.4 # ratio of meetings the model is trained on
LSTM_units = 20

model = Sequential()
model.add(Bidirectional(LSTM(LSTM_units, return_sequences=True), input_shape=(n_timesteps, 5)))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#train the model
history=model.fit(X_train, Y_train,
                  batch_size=batch_size,
                  epochs=100,
                  validation_data=[X_test, Y_test])

#measure the performance
predictions = model(X_test).numpy()

Epoch 1/100
729/729 [==============================] - 8s 6ms/step - loss: 1.0956e-09 - accuracy: 0.5449 - val_loss: 1.1868e-09 - val_accuracy: 0.5112
Epoch 2/100
729/729 [==============================] - 3s 4ms/step - loss: 1.0956e-09 - accuracy: 0.5449 - val_loss: 1.1868e-09 - val_accuracy: 0.5112
Epoch 3/100
729/729 [==============================] - 3s 4ms/step - loss: 1.0956e-09 - accuracy: 0.5449 - val_loss: 1.1868e-09 - val_accuracy: 0.5112
Epoch 4/100
729/729 [==============================] - 3s 5ms/step - loss: 1.0956e-09 - accuracy: 0.5449 - val_loss: 1.1868e-09 - val_accuracy: 0.5112
Epoch 5/100
729/729 [==============================] - 4s 5ms/step - loss: 1.0956e-09 - accuracy: 0.5449 - val_loss: 1.1868e-09 - val_accuracy: 0.5112
Epoch 6/100
729/729 [==============================] - 3s 4ms/step - loss: 1.0956e-09 - accuracy: 0.5449 - val_loss: 1.1868e-09 - val_accuracy: 0.5112
Epoch 7/100
729/729 [==============================] - 4s 5ms/step - loss: 1.0956e-09 - accura

In [59]:
Y_test_eval = Y_test[:, 1, 0]
pred_eval = predictions[:, 1, 0]
# I'll just be lazy for pred_eval, if bigger or equal to 0.5 than it's 1, else it's 0
pred_eval = (pred_eval >= 0.5).astype(float)

In [61]:
windiff = get_windiff(Y_test_eval, pred_eval)
pk = get_pk(Y_test_eval, pred_eval, k=58)
k_k = get_k_kappa(Y_test_eval, pred_eval)
print("windiff:", windiff)
print("pk:", pk)
print("K_k ", k_k)

windiff: 0.9981115744747816
pk: 0.5884251968503937
K_k  0.0005096056511108614
